In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from lib.util import *

plt.style.use("ggplot")

In [3]:
df = pd.read_json("../data/preprocessed/preprocessed.json", orient="index")
with open("../data/preprocessed/param.json", "r") as f:
    params = json.load(f)
    print("params:", params)

user_n, item_n = params["user_n"], params["item_n"]

params: {'user_n': 1096, 'item_n': 3664}


In [ ]:
uplift_based_nn_model, losses, results = train_nn(
    df=df,
    user_n=user_n,
    item_n=item_n,
    args=Args(batch_size=1000, epochs=5000, model="nn", lmda=1e-8),
)

accuracy_based_nn_model, losses, results = train_nn(
    df=df,
    user_n=user_n,
    item_n=item_n,
    args=Args(
        batch_size=1000,
        epochs=5000,
        sampling="accuracy-based-pointwise",
        model="nn",
        lmda=1e-8,
    ),
)

In [14]:
uplift_based_mf_model, losses, results = train_nn(
    df=df,
    user_n=user_n,
    item_n=item_n,
    args=Args(batch_size=1000, epochs=5000, model="mf", lmda=1e-8),
)

accuracy_based_mf_model, losses, results = train_nn(
    df=df,
    user_n=user_n,
    item_n=item_n,
    args=Args(
        batch_size=1000,
        epochs=5000,
        sampling="accuracy-based-pointwise",
        model="mf",
        lmda=1e-8,
    ),
)

Args(exp_name=None, mode='valid', model='mf', sampling='uplift-based-pointwise', seed=0, alpha=0.6, gamma_p=0.2, gamma_r=0.5, eta=0.01, lmda=1e-08, d=100, batch_size=1000, eval_step=1000, epochs=5000, device='cpu')


100%|██████████| 1096/1096 [00:01<00:00, 1042.62it/s]


{'Accuracy@10': 0.017573696145124718, 'Uplift@10': 0.02185329158118274, 'Accuracy@30': 0.010342417889587702, 'Uplift@30': 0.009329523461187424, 'Accuracy@100': 0.00669606512890095, 'Uplift@100': 0.005742563617488554}


100%|██████████| 1096/1096 [00:01<00:00, 1058.07it/s]


{'Accuracy@10': 0.04450800915331808, 'Uplift@10': 0.08089835094983831, 'Accuracy@30': 0.024139756484912653, 'Uplift@30': 0.031699100671614806, 'Accuracy@100': 0.013318538750426767, 'Uplift@100': 0.01695021479510301}


...

{'Accuracy@10': 0.09807692307692308, 'Uplift@10': 0.026869256474519647, 'Accuracy@30': 0.05751440329218107, 'Uplift@30': 0.02102907103232581, 'Accuracy@100': 0.03250815394651011, 'Uplift@100': 0.013223100854302557}


100%|██████████| 1096/1096 [00:01<00:00, 1010.40it/s]


{'Accuracy@10': 0.11814516129032257, 'Uplift@10': 0.05467509920634924, 'Accuracy@30': 0.06778435239973701, 'Uplift@30': 0.02615848377226902, 'Accuracy@100': 0.03814923427826653, 'Uplift@100': 0.014612522585775058}


100%|██████████| 1096/1096 [00:01<00:00, 996.00it/s] 


{'Accuracy@10': 0.12441977800201816, 'Uplift@10': 0.05890955103871345, 'Accuracy@30': 0.07294852820259826, 'Uplift@30': 0.026303991165494426, 'Accuracy@100': 0.04070730117340287, 'Uplift@100': 0.01506724779020716}


In [ ]:
k = 30

for model in [uplift_based_mf_model, accuracy_based_mf_model]:
    model.eval()
    s = []
    for u in tqdm.tqdm(range(user_n)):
        y = model.forward(
            u=torch.LongTensor([u] * item_n),
            i=torch.arange(item_n),
        )
        rec_list = y.argsort().cpu().detach().numpy()[::-1][:k].tolist()
        embeddings = accuracy_based_mf_model.item_embedding.forward(
            torch.LongTensor(rec_list)
        ).cpu().detach().numpy()

        d = 0
        for i in range(k):
            for j in range(k):
                d += np.linalg.norm(embeddings[i] - embeddings[j])
        d /= k * k
        s.append(d)

    plt.hist(s)
    plt.show()
    print(sum(s) / len(s))